In [ ]:
import geopandas as gpd
import os
from rtree import index

# === PATHS ===
base_dir = r"C:\Users\natda\OneDrive - Northeastern University\Desktop\NatDave\Academics\PhD_NU\RESEARCH\Traffic_Stress\Boston"
roads_path_1 = os.path.join(base_dir, "street_network.shp")
roads_path_2 = os.path.join(base_dir, "boston_streets.shp")

# === LOAD AND PREPARE DATA ===
roads1 = gpd.read_file(roads_path_1)
roads2 = gpd.read_file(roads_path_2)

# === BUILD SPATIAL INDEX FOR ROADS2 ===
idx = index.Index()
for i, geom in enumerate(roads2.geometry):
    idx.insert(i, geom.bounds)

# === FIND MATCHING GEOMETRIES AND ASSIGN BIKE_TYPE ===
def get_bike_type(geom, idx, roads2):
    candidates = [i for i in idx.intersection(geom.bounds)]
    for i in candidates:
        if geom.equals_exact(roads2.geometry.iloc[i], tolerance=100):
            return roads2.iloc[i]['bike_type']  # Return the bike_type of the matched geometry
    return None  # If no match is found, return None

# Assign bike_type values from roads2 to roads1 based on matching geometries
roads1['bike_type'] = roads1.geometry.apply(lambda g: get_bike_type(g, idx, roads2))

roads1[['bike_type']].value_counts(dropna=False)

bike_type                   
0                               16465
NaN                             10145
Off-Road Path                    5212
Proposed                         2774
Bike Lane                        1186
Sharrow                           626
Buffered Bike                     274
Bus bike lane                      46
Walk only                          20
Priority share                     18
Cycletrack- bike lane hybrid       13
Cycle track                         6
Climbing lane                       4
climbing lane                       3
pocket_LTS3                         3
Bike Lane one side                  1
Name: count, dtype: int64

In [2]:
roads2[['bike_type']].value_counts(dropna=False)

bike_type                   
0                               17578
NaN                              7550
Off-Road Path                    6603
Proposed                         2854
Bike Lane                        1226
Sharrow                           659
Buffered Bike                     262
Bus bike lane                      42
Walk only                          25
Priority share                     19
Cycletrack- bike lane hybrid       14
Cycle track                         9
pocket_LTS3                         6
Climbing lane                       5
Bike Lane one side                  2
climbing lane                       2
Buffered bike                       1
Pocket lane                         1
Name: count, dtype: int64

In [4]:
# Save the updated crossings shapefile
roads1.to_file(roads_path_1, driver='ESRI Shapefile')